In [56]:
import pdfplumber
import pandas as pd
import re
import numpy as np
import unicodedata

In [47]:
def format_data(df):
    df['completo'] = pd.Series([False]*len(df), dtype='bool')  # default False
    df['mes_cursado'] = pd.Series([pd.NaT]*len(df), dtype='datetime64[ns]')
    df['nota'] = pd.Series([np.nan]*len(df), dtype='float')
    df['clave'] = df['clave'].astype(str)
    df['asignatura'] = df['asignatura'].astype(str)
    df['credito'] = df['credito'].astype(int)
    df['pre_req'] = df['pre_req'].astype(str)
    empty_req = df['pre_req'] == '-'
    df.loc[empty_req, 'pre_req'] = "" 
    return df
    



pattern = r"([A-Z]{3,}-\d{3})\s+(.+?)\s+(\d{1,2})\s+([A-Z\-0-9,]*)"

data = []

with pdfplumber.open("/media/oa/Disk D/UNICARIBE/PENSUM-Ingenieria-de-Datos-e-Inteligencia-Organizacional.pdf") as pdf:
    full_text = ""
    for page in pdf.pages:
        full_text += page.extract_text() + "\n"

    matches = re.findall(pattern, full_text)

for match in matches:
    clave, nombre, credito,pre_req = match
    data.append({
        "clave" : clave.strip(),
        "asignatura": nombre.strip(),
        "credito" : credito.strip(),
        "pre_req" : pre_req.strip()
    })

df = pd.DataFrame(data)
df = format_data(df)
df

,clave,asignatura,credito,pre_req,completo,mes_cursado,nota
0,FGC-101,ORIENTACIÓN ACADÉMICA INSTITUCIONAL,2,,False,NaT,NaN
1,FGC-102,MÉTODO DEL TRABAJO ACADÉMICO,2,,False,NaT,NaN
2,FGC-103,METODOLOGÍA DE LA INVESTIGACIÓN,3,,False,NaT,NaN
3,ADE-101,ADMINISTRACION I,3,,False,NaT,NaN
4,FGC-104,LENGUA ESPAÑOLA I,3,FGC-102,False,NaT,NaN
5,FGC-105,MATEMÁTICA BÁSICA I,3,FGC-102,False,NaT,NaN
6,FGC-106,TECNOLOGÍA DE LA INFORMACIÓN Y COMUNICACIÓN I,3,,False,NaT,NaN
7,ING-101,INTRODUCCIÓN A LA INGENIERÍA,3,,False,NaT,NaN
8,FGC-107,HISTORIA SOCIAL DOMINICANA,3,FGC-102,False,NaT,NaN
9,FGC-108,INGLÉS I,3,FGC-102,False,NaT,NaN


In [59]:
def remove_accents(input_str):
        # Normaliza a forma decomposed (NFD), separa letras de acentos y elimina los acentos
        nfkd_form = unicodedata.normalize('NFD', input_str)
        only_ascii = "".join([c for c in nfkd_form if not unicodedata.combining(c)])
        return only_ascii


def add_grade(subject, grade: float):
    try:
        df['asignatura'] = df['asignatura'].apply(remove_accents)
        matches = df[df['asignatura'].str.contains(subject, case=False, na=False)]
        if matches.empty:
            print(f"No match found for: {subject}")
        elif len(matches) > 1:
           print(f"Multiple matches found for '{subject}': {[m for m in matches['asignatura']]}")
        # aplicar solo a la primera coincidencia:
        idx = matches.index[0]
        df.at[idx, 'nota'] = grade
        print(f"Assigned grade {grade} to subject: {df.at[idx, 'asignatura']}")
    except Exception as e:
        print(f"Error assigning grade: {e}")

add_grade('metodo del trabajo academico', 90)

Assigned grade 90 to subject: METODO DEL TRABAJO ACADEMICO


In [60]:
df.head()

,clave,asignatura,credito,pre_req,completo,mes_cursado,nota
0,FGC-101,ORIENTACION ACADEMICA INSTITUCIONAL,2,,False,NaT,NaN
1,FGC-102,METODO DEL TRABAJO ACADEMICO,2,,False,NaT,90.0
2,FGC-103,METODOLOGIA DE LA INVESTIGACION,3,,False,NaT,NaN
3,ADE-101,ADMINISTRACION I,3,,False,NaT,NaN
4,FGC-104,LENGUA ESPANOLA I,3,FGC-102,False,NaT,NaN
